In [0]:
# cores in cluster
#spark.sparkContext.defaultParallelism
sc.defaultParallelism



Out[2]: 8

In [0]:
# partitions of data -> small sets of data 
df = spark.table('dev_waypoint_customer_cluster.person_store')

df.rdd.getNumPartitions()

Out[5]: 6

In [0]:
# marking job for spark ui
spark.conf.set("JobDescription", "Step A-S: Basic initialization")

## Delete partition from Data Lake table

In [0]:
def delete_partitions(table_name, container="bronze"):
  """
    description:
      Runs a recursive deletion of bronze data except for the last day
      
    parameters:    
      table_name (the name of the table whose partition will be deleted)
      container (bronze, silver or gold)
  """
    
  folder_to_keep = "raw_date=" + str(date.today().strftime("%Y-%m-%d")) + "/"                                    # most recent data load

  files = dbutils.fs.ls(f"abfss://{container}@{storage_name}.dfs.core.windows.net/{cluster_path}/{table_name}/") # returns path, partition name and size
  path_with_partitions_list = [partition[0] for partition in files]                                              # returns list of paths 
  for path in path_with_partitions_list:                                                                         
    if folder_to_keep not in path:
      dbutils.fs.rm(path, recurse=True)                                                                          # if the path does not contain the most recent partition, delete it
      print(path + " SUCCESSFULLY DELETED")

In [0]:
spark.conf.get("spark.sql.shuffle.partitions")

Out[6]: '200'

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.get("spark.sql.shuffle.partitions")

Out[8]: '8'

In [0]:
%run /helpers/GdprUtils

In [0]:
dbutils.widgets.text('environment', 'dev')
environment = dbutils.widgets.get('environment')

print(f'The current envirement is: {environment}')

The current envirement is: dev

In [0]:
person_events_table = f'{environment}_waypoint_customer_cluster.person_events'
person_store_table = f'{environment}_waypoint_customer_cluster.person_store'

In [0]:
person_events = spark.table(person_events_table)
person_store = spark.table(person_store_table)

False

## obtain the current data by new data

In [0]:
new_silver_data = silver_data.union(bronze_data).subtract(silver_data.intersect(bronze_data))      # temp version. depends on if we need to keep duplicate records with different TimeStamp

### Schema changing  based on other DF

In [0]:
person_store = spark.createDataFrame(person_store.rdd, person_events.schema)
person_store.repartition("CountryCode").write.format('delta').mode('overwrite').option("overwriteSchema", "true").saveAsTable(person_store_table)

## sql create table based on location

In [0]:
%sql
CREATE TABLE IF NOT EXISTS raw_dev_waypoint_customer_cluster.customer_epol_events USING DELTA LOCATION "abfss://silver@aisoisidatalakedev.dfs.core.windows.net/waypoint/customer_cluster/customer_epol_events"

### Getting difference between to dataframes' schema

In [0]:
diff = lambda l1,l2: [x for x in l1 if x not in l2]
diff(person_events_std_old.schema.fields, person_events_std.schema.fields)

In [0]:
clb = ColumnListBuilder("customer_person_events", True)
cols = clb.BuildColumnListCL1()

raw_person_events = spark.table(raw_person_events_table) \
                                                    .withColumn( 'enqueuedTime', unix_timestamp(col("EnqueuedTimeUtc"),"M/d/y h:m:s a") ) \
                                                    .withColumnRenamed('paritition', 'partition')

new_person_events_std = parseJSONEvents(raw_person_events, personSchema).selectExpr(cols).where("Id IS NOT NULL")
old_person_events_std = parseJSONEvents(raw_person_events, personSchemaOld).where("Id IS NOT NULL")\
                                                                           .withColumn("CurrentAddress", col('currentAddress').cast(currentAddressSchema))\
                                                                           .withColumn("Name", col('Name').cast(nameSchema))\
                                                                           .selectExpr(cols)

person_events_std = new_person_events_std.union(old_person_events_std)

person_events_std.repartition("CountryCode").write.format('delta').option('overwriteSchema', True).mode('overwrite').saveAsTable(person_store_table)

In [0]:
clb = ColumnListBuilder("customer_company_events", True)
cols = clb.BuildColumnListCL1()

raw_company_events = spark.table(raw_company_events_table) \
                                                  .withColumn( 'enqueuedTime', unix_timestamp(col("EnqueuedTimeUtc"),"M/d/y h:m:s a") ) \
                                                  .withColumnRenamed('paritition', 'partition')
company_events_std = parseJSONEvents(raw_company_events, companySchema).selectExpr(cols)

company_events_std.repartition("CountryCode").write.format('delta').mode('overwrite').saveAsTable(company_store_table)

In [0]:
def convert_raw_to_standardized(dataset_name, raw_table_name, customer):
  """
  dataset_name = 
  raw_table_name = 
  customer = "person"
  """
  dataset_name = f'customer_{customer}_events'
  clb = ColumnListBuilder(dataset_name, True)
  cols = clb.BuildColumnListCL1()
  
  if customer == "person":
    raw_person_events = spark.table(raw_table_name) \
                                        .withColumn( 'enqueuedTime', unix_timestamp(col("EnqueuedTimeUtc"),"M/d/y h:m:s a") ) \
                                        .withColumnRenamed('paritition', 'partition')
    
    new_person_events_std = parseJSONEvents(raw_person_events, personSchema).selectExpr(cols).where("Id IS NOT NULL")
    old_person_events_std = parseJSONEvents(raw_person_events, personSchemaOld).where("Id IS NOT NULL")\
                                                                           .withColumn("CurrentAddress", col('currentAddress').cast(currentAddressSchema))\
                                                                           .withColumn("Name", col('Name').cast(nameSchema))\
                                                                           .selectExpr(cols)
    person_events_std = new_person_events_std.union(old_person_events_std)
    person_events_std.repartition("CountryCode").write.format('delta').option('overwriteSchema', True).mode('overwrite').saveAsTable(person_store_table)
    
  elif customer == "company":
    raw_company_events = spark.table(raw_table_name) \
                                                  .withColumn( 'enqueuedTime', unix_timestamp(col("EnqueuedTimeUtc"),"M/d/y h:m:s a") ) \
                                                  .withColumnRenamed('paritition', 'partition')
    company_events_std = parseJSONEvents(raw_company_events, companySchema).selectExpr(cols)

    company_events_std.repartition("CountryCode").write.format('delta').mode('overwrite').saveAsTable(company_store_table)
    
  else:
    raise ValueError("Please, specify the customer type: person or company.")

In [0]:
convert_raw_to_standardized("a", "b", "petya")

## Get the size of parquet file

In [0]:
def get_table_size(path):
  total_size = 0
  
  for folder in dbutils.fs.ls(path):
    for file in dbutils.fs.ls(path + str(folder[-2])):
      total_size += file[-1]
  return f"The parquet table is {total_size / 1000000} MB"



In [0]:
get_table_size("abfss://bronze@aisoisidatalakedev.dfs.core.windows.net/waypoint/policy/co_Cover/")

Out[2]: 'The parquet table is 246.197778 MB'

In [0]:
path = "dbfs:/dev_waypoint_policy/ex_Exposure"
dbutils.fs.ls("dbfs:/")

Out[22]: [FileInfo(path='dbfs:/FileStore/', name='FileStore/', size=0),
 FileInfo(path='dbfs:/cluster-logs/', name='cluster-logs/', size=0),
 FileInfo(path='dbfs:/databricks/', name='databricks/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0),
 FileInfo(path='dbfs:/databricks-results/', name='databricks-results/', size=0),
 FileInfo(path='dbfs:/delta/', name='delta/', size=0),
 FileInfo(path='dbfs:/dwtestdlstor/', name='dwtestdlstor/', size=0),
 FileInfo(path='dbfs:/hive_2_3_7_metastore_jar/', name='hive_2_3_7_metastore_jar/', size=0),
 FileInfo(path='dbfs:/hive_3_1_0_metastore_jar/', name='hive_3_1_0_metastore_jar/', size=0),
 FileInfo(path='dbfs:/hive_metastore_jar/', name='hive_metastore_jar/', size=0),
 FileInfo(path='dbfs:/ml/', name='ml/', size=0),
 FileInfo(path='dbfs:/mnt/', name='mnt/', size=0),
 FileInfo(path='dbfs:/pipelines/', name='pipelines/', size=0),
 FileInfo(path='dbfs:/tmp/', name='tmp/', size=0),
 FileInfo(path='dbfs:/tmp_copy_location/', name='tmp_copy_location/', size=0),
 FileInfo(path='dbfs:/user/', name='user/', size=0)]

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/")

Out[24]: [FileInfo(path='dbfs:/user/hive/warehouse/ampolicyfrozenmonth_testdata/', name='ampolicyfrozenmonth_testdata/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bisnode.db/', name='bisnode.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bisnode_standardized.db/', name='bisnode_standardized.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_e3p.db/', name='bronze_e3p.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_kriand.db/', name='bronze_kriand.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_linlae.db/', name='bronze_linlae.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_makval.db/', name='bronze_makval.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_medrai.db/', name='bronze_medrai.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_nilfri.db/', name='bronze_nilfri.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_oh8.db/', name='bronze_oh8.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_rudgin.db/', name='bronze_rudgin.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/bronze_serjek.db/', name='bronze_serjek.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/cdw004_dwv.db/', name='cdw004_dwv.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/cdw_jdbc_test.db/', name='cdw_jdbc_test.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/cdw_top5_jdbc_test.db/', name='cdw_top5_jdbc_test.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/cdw_waypointdw.db/', name='cdw_waypointdw.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/compass.db/', name='compass.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/crm.db/', name='crm.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/customertest/', name='customertest/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_anirudh.db/', name='dev_anirudh.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_anirudh_seed.db/', name='dev_anirudh_seed.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_cdw_cdwv_dev_cdw_dwv.db/', name='dev_cdw_cdwv_dev_cdw_dwv.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_cdw_cdwv_gdpr.db/', name='dev_cdw_cdwv_gdpr.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_cherniaev.db/', name='dev_cherniaev.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_cherniaev_seed.db/', name='dev_cherniaev_seed.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_enigma.db/', name='dev_enigma.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_gdpr.db/', name='dev_gdpr.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_google.db/', name='dev_google.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_lars.db/', name='dev_lars.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_lars_seed.db/', name='dev_lars_seed.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_valdas.db/', name='dev_valdas.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_valdas_seed.db/', name='dev_valdas_seed.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_waypoint_dw.db/', name='dev_waypoint_dw.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dev_waypointdw.db/', name='dev_waypointdw.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dim_new/', name='dim_new/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/dim_sk/', name='dim_sk/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/enigma.db/', name='enigma.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/env_enigma.db/', name='env_enigma.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/gdpr.db/', name='gdpr.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/google.db/', name='google.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/mybusiness.db/', name='mybusiness.db/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/octopus.db/', name='octopus.db/', size=0),
 FileInfo(path='dbfs:/user/hive

In [0]:
dbutils.fs.ls("dbfs:/databricks-datasets/learning-spark-v2/us_population.json")

Out[33]: [FileInfo(path='dbfs:/databricks-datasets/learning-spark-v2/us_population.json', name='us_population.json', size=3299821)]

In [0]:
files = dbutils.fs.ls("abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/")
display(files)

path,name,size
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/accounting/,accounting/,0
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/agreements/,agreements/,0
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/claims/,claims/,0
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/customer/,customer/,0
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/customer_cluster/,customer_cluster/,0
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/polaris/,polaris/,0
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/policy/,policy/,0
abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/rating/,rating/,0


In [0]:
files = dbutils.fs.ls("abfss://bronze@aisoisidatalanding.dfs.core.windows.net/")
display(files)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-953781962595124> in <module> 
 ----> 1 files = dbutils . fs . ls ( "abfss://bronze@aisoisidatalanding.dfs.core.windows.net/" ) 
 2 display ( files ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 328 exc . __context__ = None 
 329 exc . __cause__ = None 
 --> 330 raise exc
 331 return f_with_exception_handling
 332 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: Operation failed: "The specified filesystem does not exist.", 404, GET, https://aisoisidatalanding.dfs.core.windows.net/bronze?upn=false&resource=filesystem&maxResults=500&timeout=90&recursive=false, FilesystemNotFound, "The specified filesystem does not exist. RequestId:f9e010f4-301f-0089-1993-8765a2000000 Time:2021-08-02T11:41:23.3716944Z"
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.checkException(AzureBlobFileSystem.java:1179)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.listStatus(AzureBlobFileSystem.java:404)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:154)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:153)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: Operation failed: "The specified filesystem does not exist.", 404, GET, https://aisoisidatalanding.dfs.core.windows.net/bronze?upn=false&resource=filesystem&maxResults=500&timeout=90&recursive=false, FilesystemNotFound, "The specified filesystem does not exist. RequestId:f9e010f4-301f-0089-1993-8765a2000000 Time:2021-08-02T11:41:23.3716944Z"
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsRestOperation.execute(AbfsRestOperation.java:237)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsClient.listPath(AbfsClient.java:234)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.listStatus(AzureBlobFileSystemStore.java:916)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.listStatus(AzureBlobFileSystemStore.java:878)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.listStatus(AzureBlobFileSystem.java:401)
	... 15 more

In [0]:
print(int(5e8))


500000000

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold","500m")

In [0]:
files = dbutils.fs.ls("abfss://bronze@aisoisidatalanding.blob.core.windows.net/")
display(files)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-4120248666167295> in <module> 
 ----> 1 files = dbutils . fs . ls ( "abfss://bronze@aisoisidatalanding.blob.core.windows.net/" ) 
 2 display ( files ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 328 exc . __context__ = None 
 329 exc . __cause__ = None 
 --> 330 raise exc
 331 return f_with_exception_handling
 332 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: Operation failed: "The specified container does not exist.", 404, GET, https://aisoisidatalanding.blob.core.windows.net/bronze?upn=false&resource=filesystem&maxResults=500&timeout=90&recursive=false, , ""
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.checkException(AzureBlobFileSystem.java:1179)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.listStatus(AzureBlobFileSystem.java:404)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:154)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:153)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: Operation failed: "The specified container does not exist.", 404, GET, https://aisoisidatalanding.blob.core.windows.net/bronze?upn=false&resource=filesystem&maxResults=500&timeout=90&recursive=false, , ""
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsRestOperation.execute(AbfsRestOperation.java:237)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsClient.listPath(AbfsClient.java:234)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.listStatus(AzureBlobFileSystemStore.java:916)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.listStatus(AzureBlobFileSystemStore.java:878)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.listStatus(AzureBlobFileSystem.java:401)
	... 15 more

In [0]:
%fs ls 

In [0]:
files = dbutils.fs.ls("waypoint-customer-cluster/customer_epol/")
display(files)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-403365628333441> in <module> 
 ----> 1 files = dbutils . fs . ls ( "waypoint-customer-cluster/customer_epol/" ) 
 2 display ( files ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 328 exc . __context__ = None 
 329 exc . __cause__ = None 
 --> 330 raise exc
 331 return f_with_exception_handling
 332 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: File /6540411196725096/waypoint-customer-cluster/customer_epol does not exist.
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.listStatus(NativeAzureFileSystem.java:2387)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:434)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:455)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:279)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:271)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:455)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:415)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:341)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:455)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:150)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:154)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:153)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
test = spark.table("dev_puzzel.agents").limit(5)
test.display()

agent_id,customer_key,user_name,user_num,full_name,usergroup_id,usergroup_name,email,mobile,dte_updated,chat_role,chat_master_user_id,unblockable_role,unblockable_group,deleted,ExtractDate
16598,10076,A8Y,20968,Anita Elvbakken,25279,Trondheim Outbound 4,,,2018-08-30T17:00:01.170+0000,null,null,null,null,false,2018-08-31T01:00:12.053+0000
18333,100761,MN7,95557,Margareth Nordgård,10898,MO Sarpsborg,margareth.nordgard@if.no,,2018-08-30T16:00:05.677+0000,null,null,null,null,false,2018-08-31T01:00:12.053+0000
18338,100761,RIJ,95562,Rigmor Jensen,10885,ED Skadesenter Nord,rigmor.jensen@if.no,,2018-08-30T10:00:00.973+0000,0,18338,null,null,false,2018-08-31T01:00:12.053+0000
18366,100761,B5L,95594,Berit Linga,10889,ED Skadesenter Sør,berit.linga@if.no,,2018-08-30T15:00:01.790+0000,null,null,null,null,false,2018-08-31T01:00:12.053+0000
18370,100761,SNK,95598,Sigrun Kjellevand,10889,ED Skadesenter Sør,sigrun.kjellevand@if.no,,2018-08-30T15:00:01.790+0000,null,null,null,null,false,2018-08-31T01:00:12.053+0000


In [0]:
test.select(col("full_name").split(" ")).limit(5).display()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-953781962595054> in <module> 
 ----> 1 test . select ( col ( "full_name" ) . split ( " " ) ) . limit ( 5 ) . display ( ) 

 NameError : name 'col' is not defined

In [0]:
partition_list = [partition[0] for partition in spark.sql("SHOW PARTITIONS RAW_DEV_WAYPOINT_POLICY.CO_COVER").collect()]
for i in partition_list:
  print(i)

raw_date=2021-04-15
raw_date=2021-07-12

In [0]:
dbutils.fs.ls("abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/customer_cluster/customer_epol")

Out[1]: [FileInfo(path='abfss://bronze@aisoisidatalake.dfs.core.windows.net/waypoint/customer_cluster/customer_epol/CustEPol_2021-08-11.csv', name='CustEPol_2021-08-11.csv', size=41003)]

In [0]:
from datetime import date, timedelta

lag_date = date.today() + timedelta(days=-2)
lag_date

Out[8]: datetime.date(2021, 8, 10)